# Ratings_cleaning musi zostać uruchomiony jako pierwszy
Notebook `Ratings_cleaning.ipynb` usuwa niektóre z wystawionych ocen, co zmniejsza ich liczbę u użytkowników. Następnie usuwa użytkowników, którzy mają mniej niż 20 ocen i zapisuje nowy zbiór danych do pliku `user_features_2.csv`.

In [15]:
import pandas as pd

In [16]:
df_train = pd.read_csv('../data/user_features_train_2.csv')
df_test = pd.read_csv('../data/user_features_test_2.csv')

In [17]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125768 entries, 0 to 125767
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 125768 non-null  int64  
 1   num_rating             125768 non-null  int64  
 2   avg_rating             125768 non-null  float64
 3   weekend_watcher        125768 non-null  int64  
 4   type_of_viewer         125768 non-null  object 
 5   genre_Action           125316 non-null  float64
 6   genre_Adventure        125428 non-null  float64
 7   genre_Animation        113546 non-null  float64
 8   genre_Comedy           125576 non-null  float64
 9   genre_Crime            125026 non-null  float64
 10  genre_Documentary      54218 non-null   float64
 11  genre_Drama            125747 non-null  float64
 12  genre_Family           119939 non-null  float64
 13  genre_Fantasy          124296 non-null  float64
 14  genre_History          114718 non-nu

# Uzupełnienie brakujących ocen 

In [18]:
genre_columns = ['genre_Action','genre_Adventure','genre_Animation','genre_Comedy','genre_Crime','genre_Documentary','genre_Drama','genre_Family','genre_Fantasy','genre_History','genre_Horror','genre_Musical','genre_Mystery','genre_Romance','genre_Science Fiction','genre_TV Movie','genre_Thriller','genre_War','genre_Western']

for col in genre_columns:
    df_train[col] = df_train[col].fillna(df_train['avg_rating'])
    df_test[col] = df_test[col].fillna(df_test['avg_rating'])

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125768 entries, 0 to 125767
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 125768 non-null  int64  
 1   num_rating             125768 non-null  int64  
 2   avg_rating             125768 non-null  float64
 3   weekend_watcher        125768 non-null  int64  
 4   type_of_viewer         125768 non-null  object 
 5   genre_Action           125768 non-null  float64
 6   genre_Adventure        125768 non-null  float64
 7   genre_Animation        125768 non-null  float64
 8   genre_Comedy           125768 non-null  float64
 9   genre_Crime            125768 non-null  float64
 10  genre_Documentary      125768 non-null  float64
 11  genre_Drama            125768 non-null  float64
 12  genre_Family           125768 non-null  float64
 13  genre_Fantasy          125768 non-null  float64
 14  genre_History          125768 non-nu

# Enkodowanie kolumny 'type_of_viewer'

In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

def encoder(df):
    encoded = ohe.fit_transform(df[['type_of_viewer']])
    encoded = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(['type_of_viewer']))
    encoded.index = df.index
    df = df.drop(columns=['type_of_viewer'])
    df = pd.concat([df, encoded], axis=1)

    return df

df_train = encoder(df_train)
df_test = encoder(df_test)

print(df_train.head(3))
print(df_test.head(3))


   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       1         141    3.531915                0      4.133333   
1       2          52    4.269231                0      4.000000   
2       3         147    3.588435                0      3.583333   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0         3.333333         3.531915      3.339623     4.285714   
1         4.142857         4.875000      4.130435     4.000000   
2         3.605263         3.961538      3.186275     3.346154   

   genre_Documentary  ...  genre_Mystery  genre_Romance  \
0           3.000000  ...       4.272727       3.487805   
1           4.269231  ...       4.750000       4.823529   
2           3.588435  ...       3.875000       3.637931   

   genre_Science Fiction  genre_TV Movie  genre_Thriller  genre_War  \
0               4.000000        3.531915        3.666667   3.555556   
1               4.269231        4.269231        4.300000   3.000000   
2           

# Dołączenie ostatnich 20 ocenionych filmów ze zbioru ratings_clean_zlib.h5

In [20]:
ratings_df = pd.read_parquet('../data/ratings_clean_groupped_20.parquet')
df_train = pd.merge(df_train, ratings_df, on='userId')
df_test = pd.merge(df_test, ratings_df, on='userId')

# Normalizacja danych numerycznych

In [21]:
from sklearn.preprocessing import StandardScaler

columns_to_normalize = ['num_rating', 'avg_rating']
columns_to_normalize.extend(genre_columns)
print('Columns to normalize:', columns_to_normalize)

scaler = StandardScaler()

df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
print(df_train.head(3))

Columns to normalize: ['num_rating', 'avg_rating', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller', 'genre_War', 'genre_Western']
   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       1   -0.171417   -0.543450                0      0.840566   
1       2   -0.455905    1.083413                0      0.613863   
2       3   -0.152238   -0.418739                0     -0.094582   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0        -0.549841        -0.302503     -0.598797     0.839517   
1         0.853034         1.555059      0.872001     0.308963   
2        -0.078597         0.291690     -0.884002    -0.905189   

   genre_Documentary  ...  genre_TV Movie  genre_Thriller  genre_War  \
0         

## Dla danych testowych

In [22]:
df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])
print(df_test.head(3))

   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       7   -0.481476   -0.312987                1     -1.086407   
1      13   -0.414350   -0.291385                0     -0.012552   
2      23   -0.420743    1.487528                0      1.038931   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0        -0.694255        -0.346643     -0.167671    -0.103690   
1        -0.742393        -0.144504     -0.372051    -1.017421   
2         0.976817        -1.038171      1.443098     1.094591   

   genre_Documentary  ...  genre_TV Movie  genre_Thriller  genre_War  \
0           1.711934  ...       -0.245651       -0.234691  -0.461043   
1          -0.261627  ...       -0.228318       -0.424850   0.093484   
2           1.711934  ...        1.198995        1.271045   1.025397   

   genre_Western  type_of_viewer_negative  type_of_viewer_neutral  \
0      -0.354259                      0.0                     1.0   
1      -2.396376                   

# Konwersja kolumny weekend_watcher do float64

In [23]:
df_train['weekend_watcher'] = df_train['weekend_watcher'].astype('float64')
df_test['weekend_watcher'] = df_test['weekend_watcher'].astype('float64')

In [24]:
print('\', \''.join(df_train.columns.to_list()))

userId', 'num_rating', 'avg_rating', 'weekend_watcher', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller', 'genre_War', 'genre_Western', 'type_of_viewer_negative', 'type_of_viewer_neutral', 'type_of_viewer_positive', 'movies_seq', 'ratings_seq', 'ts_seq


# Zapis do pliku

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125768 entries, 0 to 125767
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   125768 non-null  int64  
 1   num_rating               125768 non-null  float64
 2   avg_rating               125768 non-null  float64
 3   weekend_watcher          125768 non-null  float64
 4   genre_Action             125768 non-null  float64
 5   genre_Adventure          125768 non-null  float64
 6   genre_Animation          125768 non-null  float64
 7   genre_Comedy             125768 non-null  float64
 8   genre_Crime              125768 non-null  float64
 9   genre_Documentary        125768 non-null  float64
 10  genre_Drama              125768 non-null  float64
 11  genre_Family             125768 non-null  float64
 12  genre_Fantasy            125768 non-null  float64
 13  genre_History            125768 non-null  float64
 14  genr

In [26]:
df_train.to_parquet('../data/user_features_clean_train.parquet', compression='brotli')

In [27]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31255 entries, 0 to 31254
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   userId                   31255 non-null  int64  
 1   num_rating               31255 non-null  float64
 2   avg_rating               31255 non-null  float64
 3   weekend_watcher          31255 non-null  float64
 4   genre_Action             31255 non-null  float64
 5   genre_Adventure          31255 non-null  float64
 6   genre_Animation          31255 non-null  float64
 7   genre_Comedy             31255 non-null  float64
 8   genre_Crime              31255 non-null  float64
 9   genre_Documentary        31255 non-null  float64
 10  genre_Drama              31255 non-null  float64
 11  genre_Family             31255 non-null  float64
 12  genre_Fantasy            31255 non-null  float64
 13  genre_History            31255 non-null  float64
 14  genre_Horror          

In [28]:
df_test.to_parquet('../data/user_features_clean_test.parquet', compression='brotli')